In [1]:
from typing import List, Union

from qiskit import QuantumRegister, QuantumCircuit
import scipy
from scipy.optimize import minimize

import tequila as tq
import numpy as np
import qiskit as qsk
from qiskit import Aer

## Question 1.
Q) Implement an error-mitigation protocol based on removing measurement results corresponding to a wrong number of
electrons, which is described in Ref. [14], (see Sec. 3.4. Post-processing Procedure). How different are the results of
simulations with and without error-mitigation?

A) For example, H<sub>2 </sub> molecule in BK transformation, the hamiltonian, number operator and spin operator are
given as below.

$ \hat{H}_{BK}(R=0.75Å) = -0.109731 $
$ \quad + 0.169885 z_0 + 0.168212 z_1 + 0.169885 z_1 z_0 $
$ \quad + 0.0454429 x_2 z_1 x_0 - 0.218863 z_2 + 0.0454429 y_2 z_1 y_0 $
$ \quad + 0.120051 z_2 z_0 + 0.165494 z_2 z_1 z_0 + 0.173954 z_3 z_1 $
$ \quad + 0.0454429 z_3 x_2 z_1 x_0 + 0.120051 z_3 z_2 z_0 $
$ \quad + 0.165494 z_3 z_2 z_1 z_0 $

In [2]:
from Week3_VQE.utility import construct_QMF_ansatz, init_qcc_params, hf_occ

H = tq.QubitHamiltonian("""
    -0.109731 + 0.169885 Z(0) + 0.168212 Z(1) + 0.169885 Z(1) Z(0)
    + 0.0454429 X(2) Z(1) X(0) - 0.218863 Z(2) + 0.0454429 Y(2) Z(1) Y(0)
    + 0.120051 Z(2) Z(0) + 0.165494 Z(2) Z(1) Z(0) + 0.173954 Z(3) Z(1)
    + 0.0454429 Z(3) Z(2) Z(1) X(0) + 0.120051 Z(3) Z(2) Z(0)
    + 0.165494 Z(3) Z(2) Z(1) Z(0)
""".replace('/n', ''))

A = tq.QubitHamiltonian("""
    0.169885 Z(0) + 0.168212 Z(1) + 0.169885 Z(1) Z(0)
    - 0.218863 Z(2) + 0.120051 Z(2) Z(0) + 0.165494 Z(2) Z(1) Z(0) + 0.173954 Z(3) Z(1)
    + 0.120051 Z(3) Z(2) Z(0) + 0.165494 Z(3) Z(2) Z(1) Z(0)
""".replace('/n', ''))

B = tq.QubitHamiltonian("""
0.0454429 X(2) Z(1) X(0) + 0.0454429 Y(2) Z(1) Y(0) + 0.0454429 Z(3) Z(2) Z(1) X(0)
""".replace('/n', ''))

$ \hat{N}_{BK} = 2 - (z_0 +z_1 z_0 + z_2 + z_3 z_2 z_1 )/2 $

$ \hat{S}_{BK}^2 = ( 6 - 3 z_1 + x_2 x_0 - x_2 z_1 x_0 + y_2 y_0 +z_2 z_0 - z_2 z_1 z_0 $
$ \quad -3 z_3 z_1 - y_2 z_1 y_0 + z_3 x_2 x_0 - z_3 x_2 z_1 x_0 + z_3 y_2 y_0 - z_3 y_2 z_1 y_0 $
$ \quad + z_3 z_2 z_0 - z_3 z_2 z_1 z_0 )/8 $

We restrict the number of electron to 2, and group the measurements with qubit-wise commuting strategy.
To calculate the projection operator $\hat{P}$, such that $ \hat{N}_{BK} \hat{P} = 2 \hat{P}$, the following should hold.

$ (z_0 +z_1 z_0 + z_2 + z_3 z_2 z_1) \hat{P} = 0 $

Solving that, we have

$ \hat{P} = |0010\rangle\langle0010| + |0100\rangle\langle0100| + |0110\rangle\langle0110| $
$ \quad + |1100\rangle\langle1100| + |1000\rangle\langle1000| + |1110\rangle\langle1110|$

first, we need to divide the hamiltonian into two parts,
$ \hat{H} = \hat{A} + \hat{B} -0.109731  $, where

$ A = 0.169885 z_0 + 0.168212 z_1 + 0.169885 z_1 z_0 $
$ \quad - 0.218863 z_2 + 0.120051 z_2 z_0 + 0.165494 z_2 z_1 z_0 + 0.173954 z_3 z_1 $
$ \quad + 0.120051 z_3 z_2 z_0 + 0.165494 z_3 z_2 z_1 z_0 $

$ B = 0.0454429 x_2 z_1 x_0 + 0.0454429 y_2 z_1 y_0 + 0.0454429 z_3 x_2 z_1 x_0$

Then when we measure $ A $, we can pick out some eigenstates orthogonal to $\hat{P}$.

First, we simulate without postprocessing.

In [3]:
a = tq.Variable("tau_0")
U = construct_QMF_ansatz(4)
U += tq.gates.ExpPauli(paulistring=tq.PauliString.from_string("X(0)Y(2)"), angle=a)
print(U)

hf_reference = hf_occ(4, 2, qubit_transf='bk')
E = tq.ExpectationValue(H=H, U=U)
initial_vals = init_qcc_params(hf_reference, E.extract_variables())
result_naive = tq.minimize(objective=E, method="BFGS", initial_values=initial_vals, tol=1.e-4)
print(result_naive.energy)

circuit: 
Rx(target=(0,), parameter=beta_0)
Rz(target=(0,), parameter=gamma_0)
Rx(target=(1,), parameter=beta_1)
Rz(target=(1,), parameter=gamma_1)
Rx(target=(2,), parameter=beta_2)
Rz(target=(2,), parameter=gamma_2)
Rx(target=(3,), parameter=beta_3)
Rz(target=(3,), parameter=gamma_3)
Exp-Pauli(target=(0, 2), control=(), parameter=tau_0, paulistring=X(0)Y(2))

Optimizer: <class 'tequila.optimizers.optimizer_scipy.OptimizerSciPy'> 
backend         : qulacs
device          : None
samples         : None
save_history    : True
noise           : None

Method          : BFGS
Objective       : 1 expectationvalues
gradient        : 18 expectationvalues

active variables : 9

E=-0.89728800  angles= {beta_0: 3.141592653589793, gamma_0: 0.0, beta_1: 0.0, gamma_1: 0.0, beta_2: 0.0, gamma_2: 0.0, beta_3: 0.0, gamma_3: 0.0, tau_0: 0.0}  samples= None
E=-0.90323123  angles= {beta_0: 3.1415926279784223, gamma_0: 0.0, beta_1: 0.0, gamma_1: 0.0, beta_2: 3.725290298461914e-09, gamma_2: 0.0, beta_3: 0.0, 

With postprocessing, we perform the simulation.

In [4]:
initial_vals_list = [initial_vals["beta_0"], initial_vals["gamma_0"],
                     initial_vals["beta_1"], initial_vals["gamma_1"],
                     initial_vals["beta_2"], initial_vals["gamma_2"],
                     initial_vals["beta_3"], initial_vals["gamma_3"],
                     initial_vals["tau_0"]]

def meas_sv(ham, sv, proj):
    energy = 0
    rej = 0
    for term in ham.keys():
        coeff = ham[term]
        f = [False for _ in range(4)]
        for i, p in term:
            if p in ['X', 'Y']:
                f[i] = True
        f = sum([2 ** i for i, v in enumerate(f) if v])
        for psi_left in range(16):
            if proj is not None:
                assert f == 0
                if psi_left not in proj:
                    rej += abs(sv[psi_left])**2
                    continue
            psi_right = psi_left ^ f
            phase = 0
            for i, p in term:
                if p == 'Z' and (psi_right >> i) & 1:
                    phase += 2
                elif p == 'Y':
                    if (psi_right >> i) & 1:
                        phase -= 1
                    else:
                        phase += 1
            tmp = coeff * (sv[psi_left].conjugate()) * sv[psi_right]
            if phase % 4 == 0:
                energy += tmp
            elif phase % 4 == 1:
                energy += 1j * tmp
            elif phase % 4 == 2:
                energy += -1 * tmp
            else:
                energy += -1j * tmp
    return energy.real, rej/len(ham)


def sim_H_with_postprocessing(vals:List):
    shots = 8096
    vals_dict = {"beta_0": vals[0], "gamma_0": vals[1],
                 "beta_1": vals[2], "gamma_1": vals[3],
                 "beta_2": vals[4], "gamma_2": vals[5],
                 "beta_3": vals[6], "gamma_3": vals[7],
                 "tau_0": vals[8]}

    qr = QuantumRegister(4)
    qc = QuantumCircuit(qr)

    # Construct QMF
    # gates.Rx(target=q0, angle=beta) + gates.Rz(target=q0, angle=gamma)
    for q in range(4):
        qc.rx(vals[2*q], q)
        qc.rz(vals[2*q + 1], q)

    # Construct QCC "X(0)Y(2)"
    qc.h(0)
    qc.rx(-np.pi / 2, 2)
    qc.cx(0, 2)
    qc.rz(2 * vals[8], 2)
    qc.cx(0, 2)
    qc.h(0)
    qc.rx(np.pi / 2, 2)

    # statevector
    sv_backend = Aer.get_backend("statevector_simulator")
    job = qsk.execute(qc, backend=sv_backend)
    final_state = job.result().get_statevector(qc)

    proj = [4, 2, 6, 3, 1, 7]
    meas_A, rej = meas_sv(A, final_state, proj)
    meas_A = meas_A / (1-rej)
    meas_B, _ = meas_sv(B, final_state, None)
    return meas_A + meas_B -0.109731

post_res = minimize(sim_H_with_postprocessing, x0=initial_vals_list, method='BFGS')
print(post_res)
print(post_res.fun)

      fun: -0.9046329654108264
 hess_inv: array([[1.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        1.68124192e-15, 0.00000000e+00, 1.68124192e-15, 0.00000000e+00,
        4.10172003e-08],
       [0.00000000e+00, 1.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00],
       [1.68124192e-15, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        1.00000000e+00, 0.00000000e+00, 2.42541417e-15, 0.00000000e+00,
        1.81840396e-08],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 1.00000000e+00, 0

In [5]:
print(f"difference = {result_naive.energy - post_res.fun}")

difference = -1.6542323066914832e-14


So we have almost zero difference, as this the simulation without postprocessing doesn't break the number symmetry.

## Question 2.
Q) Can the error-mitigation protocol described in Ref. [14] be used for more complicated symmetries, like $S^2$?

A) It's hard to find the hamiltonian part whose eigenstates are that of $\hat{S}^2$. So we need further studies.


## Question 3.
Q) Optional: Suggest an error-mitigation protocol if you know that the right wavefunction should
be an eigenstate of a certain multi-qubit operator $\hat{A}$ with eigenvalue $a$.

A)
